In [1]:
from itertools import chain

import pandas as pd

from ddeserts.annotate import RACES
from ddeserts.annotate import add_all_stat_columns
from ddeserts.annotate import add_geo_columns
from ddeserts.annotate import with_columns_sorted
from ddeserts.load import load_cvap_data
from ddeserts.load import load_charter_cities

In [2]:
# load the data
pre_filter = lambda line: ('California' in line and 'CDP' not in line)

df = load_cvap_data('Place', pre_filter=pre_filter)
len(df)

485

In [3]:
# add geo columns and post-filter
add_geo_columns(df)
df = df[(df['state'] == 'California') & df['geotype'].isin(['city', 'town'])]
len(df)

482

In [4]:
# add charter city data
charter_cities = load_charter_cities()
df['geosubtype'] = 'general law'
df.loc[df['name'].isin(charter_cities), 'geosubtype'] = 'charter'

In [5]:
# add stat columns and sort
add_all_stat_columns(df)
df = with_columns_sorted(df)

In [6]:
# sort by common name of city (Ventura, not San Buenaventura)
df.sort_values(by=['name'], inplace=True)

In [7]:
df

geoid                        geoname   geosubtype  \
table line                                                                  
Place 26573  1600000US0600296      Adelanto city, California      charter   
      26599  1600000US0600394  Agoura Hills city, California  general law   
      26664  1600000US0600562       Alameda city, California      charter   
      26690  1600000US0600674        Albany city, California      charter   
      26729  1600000US0600884      Alhambra city, California      charter   
...                       ...                            ...          ...   
      47399  1600000US0686930    Yountville city, California  general law   
      47412  1600000US0686944         Yreka city, California  general law   
      47425  1600000US0686972     Yuba City city, California  general law   
      47438  1600000US0687042       Yucaipa city, California  general law   
      47451  1600000US0687056  Yucca Valley town, California  general law   

            geotype          name       state  adu_est  adu_moe  cit_est  \
table line                                                                 
Place 26573    city      Adelanto  California    22605      761    31030   
      26599    city  Agoura Hills  California    16015      506    19225   
      26664    city       Alameda  California    62765      652    73495   
      26690    city        Albany  California    14990      317    16765   
      26729    city      Alhambra  California    69775      654    70645   
...             ...           ...         ...      ...      ...      ...   
      47399    city    Yountville  California     2755       92     2935   
      47412    city         Yreka  California     5640      298     7545   
      47425    city     Yuba City  California    49710      442    59405   
      47438    city       Yucaipa  California    41020     1119    51955   
      47451    city  Yucca Valley  California    16960      570    21120   

             cit_moe  ...  prop_his_adu_dvap_est  prop_his_adu_dvap_moe  \
table line            ...                                                 
Place 26573      555  ...               0.199111               0.074430   
      26599      362  ...               0.163539               0.434848   
      26664      855  ...               0.168443               0.114563   
      26690      503  ...               0.271137               0.189370   
      26729     1162  ...               0.136017               0.056382   
...              ...  ...                    ...                    ...   
      47399       61  ...               0.031915               0.625063   
      47412       97  ...               0.056911               0.414314   
      47425      787  ...               0.312090               0.056926   
      47438      550  ...               0.132381               0.104603   
      47451      270  ...               0.084767               0.202328   

             prop_ind_adu_dvap_est  prop_ind_adu_dvap_moe  \
table line                                                  
Place 26573               0.000000               1.000000   
      26599                    NaN                    NaN   
      26664               0.000000               0.573129   
      26690               0.000000               0.969746   
      26729               0.028169               0.567457   
...                            ...                    ...   
      47399               0.000000               1.000000   
      47412               0.000000               0.481880   
      47425               0.000000               0.277837   
      47438               0.052632               0.947368   
      47451               0.000000               0.865670   

             prop_pac_adu_dvap_est  prop_pac_adu_dvap_moe  \
table line                                                  
Place 26573               0.285714               0.714286   
      26599                    NaN                    NaN   
      26664        

In [8]:
df.to_csv('output/ca-cities.csv')

In [9]:
# general disenfranchisement data
cols = (
    ['name', 'geosubtype'] +
    list(chain(*(
        [f'{pop}_est', f'{pop}_moe'] 
        for pop in ('tot', 'adu', 'dvap', 'prop_adu_dvap')
    ))) +
    list(chain(*(
        [f'{r}_dvap_est', f'{r}_dvap_moe', 
         f'prop_{r}_adu_dvap_est', f'prop_{r}_adu_dvap_moe']
        for r in ('wht', 'his', 'asn', 'blk', 'ind', 'pac', 'tmr')
    )))
)
dg = df[cols]
dg.to_csv('output/ca-cities-dvap.csv')

In [10]:
# racial disparity data
cols = (
    ['name', 'geosubtype'] + 
    list(chain(*(
        [f'{pop}_est', f'{pop}_moe'] for pop in ('tot', 'adu', 'cvap')
    ))) +
    list(chain(*(
        [f'prop_adu_{r}_est', f'prop_cvap_{r}_est', 
         f'racial_disp_{r}_est', f'racial_disp_{r}_moe']
        for r in ('wht', 'his', 'asn', 'blk', 'ind', 'pac', 'tmr')
    )))
)
dg = df[cols]
dg.to_csv('output/ca-cities-racial-disp.csv')

In [11]:
dg = df[['name', 'geosubtype', 'racial_disp_his_est', 'racial_disp_his_moe']]
dg[dg['geosubtype'] == 'charter'].sort_values(by=['racial_disp_his_est'])[:30]

name geosubtype  racial_disp_his_est  \
table line                                                        
Place 35751           King City    charter            -0.165947   
      42719             Salinas    charter            -0.121919   
      43252          San Rafael    charter            -0.121167   
      42914           Sand City    charter            -0.120696   
      43382         Santa Maria    charter            -0.117588   
      30512             Compton    charter            -0.103871   
      42069            Richmond    charter            -0.100994   
      35179           Inglewood    charter            -0.097968   
      35166            Industry    charter            -0.094581   
      46437         Watsonville    charter            -0.094025   
      37012             Lindsay    charter            -0.090184   
      43746             Shafter    charter            -0.090162   
      27015             Anaheim    charter            -0.084703   
      43291           Santa Ana    charter            -0.081357   
      39365                Napa    charter            -0.080496   
      43304       Santa Barbara    charter            -0.077922   
      41419         Porterville    charter            -0.077724   
      37441         Los Angeles    charter            -0.077251   
      31396  Desert Hot Springs    charter            -0.074625   
      41965        Redwood City    charter            -0.074122   
      43434          Santa Rosa    charter            -0.070672   
      39885             Oakland    charter            -0.067856   
      38351              Merced    charter            -0.067040   
      43148          San Marcos    charter            -0.066611   
      45618              Tulare    charter            -0.062347   
      37350          Long Beach    charter            -0.062214   
      46229               Vista    charter            -0.061862   
      42849      San Bernardino    charter            -0.061269   
      40444            Palmdale    charter            -0.057993   
      40860            Petaluma    charter            -0.057487   

             racial_disp_his_moe  
table line                        
Place 35751             0.148865  
      42719             0.031558  
      43252             0.024259  
      42914             0.214543  
      43382             0.038189  
      30512             0.043940  
      42069             0.027528  
      35179             0.026860  
      35166             0.520931  
      46437             0.053538  
      37012             0.170380  
      43746             0.088459  
      27015             0.018308  
      43291             0.021722  
      39365             0.019671  
      43304             0.024387  
      41419             0.050803  
      37441             0.003956  
      31396             0.061401  
      41965             0.030949  
      43434             0.014619  
      39885             0.010365  
      38351             0.033155  
      43148             0.034194  
      45618             0.050230  
      37350             0.012858  
      46229             0.031850  
      42849             0.024781  
      40444             0.030573  
      40860             0.021174

In [12]:
dg = df[['name', 'geosubtype', 'racial_disp_asn_est', 'racial_disp_asn_moe']]
dg[dg['geosubtype'] == 'charter'].sort_values(by=['racial_disp_asn_est'])[:30]

name geosubtype  racial_disp_asn_est  \
table line                                                   
Place 44838      Sunnyvale    charter            -0.140617   
      43317    Santa Clara    charter            -0.125333   
      26690         Albany    charter            -0.084724   
      35257         Irvine    charter            -0.079677   
      39196  Mountain View    charter            -0.077464   
      43265      San Ramon    charter            -0.076810   
      45462       Torrance    charter            -0.065397   
      27171        Arcadia    charter            -0.063339   
      40483      Palo Alto    charter            -0.054037   
      31045        Cypress    charter            -0.052277   
      27951       Berkeley    charter            -0.052239   
      33060         Folsom    charter            -0.046244   
      26729       Alhambra    charter            -0.045912   
      30980    Culver City    charter            -0.035708   
      43187      San Mateo    charter            -0.035477   
      43057       San Jose    charter            -0.032208   
      29784       Cerritos    charter            -0.031876   
      41965   Redwood City    charter            -0.029265   
      40665       Pasadena    charter            -0.029234   
      43096    San Leandro    charter            -0.027928   
      43343     Santa Cruz    charter            -0.026553   
      28770        Burbank    charter            -0.025075   
      42927      San Diego    charter            -0.024377   
      26664        Alameda    charter            -0.023475   
      45163    Temple City    charter            -0.022732   
      37389   Los Alamitos    charter            -0.020948   
      42979  San Francisco    charter            -0.019786   
      40379  Pacific Grove    charter            -0.017502   
      42511      Roseville    charter            -0.017497   
      43395   Santa Monica    charter            -0.017406   

             racial_disp_asn_moe  
table line                        
Place 44838             0.025385  
      43317             0.027658  
      26690             0.042739  
      35257             0.023221  
      39196             0.025268  
      43265             0.033142  
      45462             0.022257  
      27171             0.049204  
      40483             0.023868  
      31045             0.034843  
      27951             0.015017  
      33060             0.017319  
      26729             0.029487  
      30980             0.027574  
      43187             0.021904  
      43057             0.006865  
      29784             0.043613  
      41965             0.021385  
      40665             0.016118  
      43096             0.029906  
      43343             0.011778  
      28770             0.016823  
      42927             0.004253  
      26664             0.023976  
      45163             0.054791  
      37389             0.033608  
      42979             0.004539  
      40379             0.018383  
      42511             0.010954  
      43395             0.013211

In [13]:
dg = df[['name', 'geosubtype', 'racial_disp_wht_est', 'racial_disp_wht_moe']]
dg[dg['geosubtype'] == 'charter'].sort_values(by=['racial_disp_wht_est'], ascending=False)[:30]

name geosubtype  racial_disp_wht_est  \
table line                                                        
Place 35751           King City    charter             0.137028   
      43252          San Rafael    charter             0.122015   
      44838           Sunnyvale    charter             0.118964   
      39196       Mountain View    charter             0.105283   
      42914           Sand City    charter             0.099033   
      43317         Santa Clara    charter             0.096673   
      43382         Santa Maria    charter             0.093367   
      41965        Redwood City    charter             0.093330   
      42719             Salinas    charter             0.083305   
      43434          Santa Rosa    charter             0.082444   
      26690              Albany    charter             0.082400   
      39365                Napa    charter             0.079162   
      43304       Santa Barbara    charter             0.078971   
      46437         Watsonville    charter             0.077753   
      37012             Lindsay    charter             0.077392   
      35166            Industry    charter             0.076847   
      43746             Shafter    charter             0.074021   
      43148          San Marcos    charter             0.071865   
      41419         Porterville    charter             0.068036   
      27015             Anaheim    charter             0.068020   
      43187           San Mateo    charter             0.067970   
      40860            Petaluma    charter             0.061412   
      46229               Vista    charter             0.060055   
      37441         Los Angeles    charter             0.059614   
      43265           San Ramon    charter             0.059006   
      35257              Irvine    charter             0.058015   
      38351              Merced    charter             0.056304   
      31396  Desert Hot Springs    charter             0.056028   
      45618              Tulare    charter             0.053775   
      40483           Palo Alto    charter             0.052693   

             racial_disp_wht_moe  
table line                        
Place 35751             0.098462  
      43252             0.044577  
      44838             0.022726  
      39196             0.034021  
      42914             0.379626  
      43317             0.027113  
      43382             0.029198  
      41965             0.037619  
      42719             0.021363  
      43434             0.021391  
      26690             0.049902  
      39365             0.025335  
      43304             0.034364  
      46437             0.043955  
      37012             0.084538  
      35166             0.395647  
      43746             0.050504  
      43148             0.038617  
      41419             0.041174  
      27015             0.016638  
      43187             0.027857  
      40860             0.035862  
      46229             0.031768  
      37441             0.003184  
      43265             0.037093  
      35257             0.023954  
      38351             0.030347  
      31396             0.058858  
      45618             0.038252  
      40483             0.037764

In [14]:
dg = df[df['name'] == 'King City']
dg[['prop_adu_his_est', 'prop_adu_his_moe', 'prop_dvap_his_est', 'prop_dvap_his_moe', 
    'prop_adu_dvap_est', 'prop_adu_dvap_moe', 'prop_his_adu_dvap_est', 'prop_his_adu_dvap_moe', 
    'adu_est', 'cvap_est', 'his_adu_est', 'his_adu_moe', 'his_cvap_est', 'his_cvap_moe']]

,,prop_adu_his_est,prop_adu_his_moe,prop_dvap_his_est,prop_dvap_his_moe,prop_adu_dvap_est,prop_adu_dvap_moe,prop_his_adu_dvap_est,prop_his_adu_dvap_moe,adu_est,cvap_est,his_adu_est,his_adu_moe,his_cvap_est,his_cvap_moe
table,line,,,,,,,,,,,,,,
Place,35751,0.849444,0.069953,0.98583,0.185025,0.548889,0.055279,0.637018,0.061523,9000,4060,7645,506,2775,433


In [15]:
dg = df[df['name'] == 'King City']
dg[['prop_adu_his_est', 'prop_cvap_his_est', 'racial_disp_his_est', 'racial_disp_his_moe']]

,,prop_adu_his_est,prop_cvap_his_est,racial_disp_his_est,racial_disp_his_moe
table,line,,,,
Place,35751,0.849444,0.683498,-0.165947,0.148865


In [16]:
df[['name', 'prop_adu_dvap_moe']]

name  prop_adu_dvap_moe
table line                                  
Place 26573      Adelanto           0.042975
      26599  Agoura Hills           0.044954
      26664       Alameda           0.017765
      26690        Albany           0.037616
      26729      Alhambra           0.017209
...                   ...                ...
      47399    Yountville           0.046413
      47412         Yreka           0.075566
      47425     Yuba City           0.019877
      47438       Yucaipa           0.039395
      47451  Yucca Valley           0.048448

[482 rows x 2 columns]

In [17]:
len(df[df['tot_est'] >= 50000])

176